In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [1]:
#from help page 540777
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(path='.azureml')

experiment_name='udacity_project_solution_real'

exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location,  
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-165760
Azure region: southcentralus
Resource group: aml-quickstarts-165760


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = 'project1cluster'

# Verify cluster does not exist
try: 
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)


    
compute_target.wait_for_completion(show_output=True)


InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
import os
# Create training directory to store training configuration files.
if "training" not in os.listdir():
    os.mkdir("./training")
    

In [4]:
#Make sure training has the most up to date version of train.py
!cp train.py ./training

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice


# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': uniform(0.0, 1.0),
        '--max_iter': choice(10, 40, 80, 100, 120, 150)
    })

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)



# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./training',
             compute_target=cpu_cluster_name,
             entry_script='train.py'
             )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=12,
                                    max_concurrent_runs=3
                                    )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

KeyError: 'log_files'

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_hyperdrive_run = hyperdrive_run.get_best_by_primary_metric()
#joblib.dump(value=best_hyperdrive_run, filename='./hyperdrive_model.pkl')

In [ ]:
from azureml.core import Model

best_hyperdrive_run.register_model(model_path='./outputs/hyperdrive_model.joblib', 
                                           model_name='best_hyperdrive_model')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

file_path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

ds = TabularDatasetFactory.from_delimited_files(path=file_path)

In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)


In [ ]:
import pandas as pd
# Combine x_train and y_train to create one training df
train = pd.concat([x_train, y_train], axis=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=train,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run

#new name?
exp = Experiment(workspace=ws, name="udacity-project-automl")

automl_run = exp.submit(automl_config)
RunDetails(remote_run).show()

In [ ]:
# Retrieve and save your best automl model.

best_automl_run, fitted_model = automl_run.get_output()

best_automl_run.register_model(model_path='./outputs/automl_model.joblib',
                               model_name='best_automl_model')